In [146]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [147]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

617

In [148]:
#Make a request for each of the cities
url = "http://api.openweathermap.org/data/2.5/weather?"
query_url = f"{url}appid={weather_api_key}&q="

import json

responses = []

for x in range(len(cities)):
    print(f"Processing Record {x} of Set 1 | {cities[x]}")
    
    response = requests.get(query_url + cities[x])
    responses.append(response.json())

Processing Record 0 of Set 1 | magdalena
Processing Record 1 of Set 1 | bambous virieux
Processing Record 2 of Set 1 | ushuaia
Processing Record 3 of Set 1 | pecos
Processing Record 4 of Set 1 | olinda
Processing Record 5 of Set 1 | hermanus
Processing Record 6 of Set 1 | cidreira
Processing Record 7 of Set 1 | chokurdakh
Processing Record 8 of Set 1 | kemise
Processing Record 9 of Set 1 | cape town
Processing Record 10 of Set 1 | butaritari
Processing Record 11 of Set 1 | vaini
Processing Record 12 of Set 1 | roebourne
Processing Record 13 of Set 1 | san patricio
Processing Record 14 of Set 1 | berlevag
Processing Record 15 of Set 1 | punta arenas
Processing Record 16 of Set 1 | catuday
Processing Record 17 of Set 1 | rikitea
Processing Record 18 of Set 1 | cabo san lucas
Processing Record 19 of Set 1 | muros
Processing Record 20 of Set 1 | busselton
Processing Record 21 of Set 1 | bluff
Processing Record 22 of Set 1 | rocha
Processing Record 23 of Set 1 | atuona
Processing Record 24 

In [151]:
with open('output_data/responses.json', 'w') as json_file:
    json.dump(responses, json_file)

In [158]:
import os
import csv
csvpath = os.path.join('output_data', 'cities1.csv')
with open (csvpath, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=" ")
    print(csvreader)
    for row in csvreader:
        print(row)

['coord/lon,coord/lat,weather/0/id,weather/0/main,weather/0/description,weather/0/icon,base,main/temp,main/feels_like,main/temp_min,main/temp_max,main/pressure,main/humidity,visibility,wind/speed,wind/deg,clouds/all,dt,sys/type,sys/id,sys/country,sys/sunrise,sys/sunset,timezone,id,name,cod,rain/1h,weather/1/id,weather/1/main,weather/1/description,weather/1/icon,wind/gust,main/sea_level,main/grnd_level,message,snow/1h,snow/3h']
['121.429,14.1999,803,Clouds,broken', 'clouds,04n,stations,299.54,303.19,299.26,300.15,1011,74,10000,1.03,0,75,1610118195,1,8160,PH,1610058044,1610098837,28800,1703912,Magdalena,200,,,,,,,,,,,']
['57.7575,-20.3428,500,Rain,light', 'rain,10n,stations,299.73,299.51,299.26,300.15,1008,74,8000,6.69,120,75,1610118195,1,2061,MU,1610069801,1610117651,14400,1106677,Bambous', 'Virieux,200,0.21,,,,,,,,,,']
['-68.3,-54.8,500,Rain,light', 'rain,10d,stations,281.15,271.56,281.15,281.15,996,76,10000,11.83,230,75,1610118061,1,8303,AR,1610093449,1610154528,-10800,3833367,Ushuaia

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 5715: character maps to <undefined>

In [ ]:
df = pd.DataFrame({
    city = response_json['sys']['name']
    lat = response_json['coord']['lat']
    lng = response_json['coord']['lon']
    max_temp = response_json['main']['temp_max']
    humidity = response_json['main']['humidity']
    cloudiness = response_json['clouds']['all']
    wind_speed = response_json['wind']['speed']
    country = response_json['sys']['country']
    date = response_json['dt']
})